In [ ]:
# ======================= IMPORTS =======================
from google import genai
from google.genai import types
from google.colab import drive

import os, json, time, random, wave, base64
from datetime import datetime
from pathlib import Path

import torch
import librosa
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2FeatureExtractor

# ======================= MOUNT DRIVE =======================
drive.mount('/content/drive')

# ======================= API KEY ROTATION =======================
KEYS_FILE = "/content/drive/MyDrive/Alot1.txt"

with open(KEYS_FILE) as f:
    API_KEYS = [k.strip() for k in f if k.strip()]

assert API_KEYS, " No API keys found!"

current_key_index = 20
client = None

def configure_key():
    global client
    client = genai.Client(api_key=API_KEYS[current_key_index])
    print(f" Using API key {current_key_index + 1}/{len(API_KEYS)}")

def rotate_key():
    global current_key_index
    current_key_index += 1
    if current_key_index >= len(API_KEYS):
        current_key_index = 0
    configure_key()

configure_key()

# ======================= PATHS =======================
DATASET_DIR = "/content/drive/MyDrive/teacher_dataset_large_updated"
VOICE_DIR = os.path.join(DATASET_DIR, "voices")
METADATA_FILE = os.path.join(DATASET_DIR, "metadata.jsonl")
os.makedirs(VOICE_DIR, exist_ok=True)

# ======================= VOICES & STYLES (MAPPED GENDER) =======================
VOICE_MAP = {
    "Puck": "male",
    "Charon": "male",
    "Orus": "male",
    "Achird": "male",
    "Enceladus": "male",
    "Zephyr": "female",
    "Leda": "female",
    "Kore": "female",
    "Aoede": "female",
    "Gacrux": "female",
    "Sulafat": "female",
}

VOICES = list(VOICE_MAP.keys())

STYLES = [
    "cheerful","gentle","energetic","whispering","authoritative",
    "playful","calm","excited","sad","serious","friendly",
    "enthusiastic","soothing","firm","formal","curious",
    "happy","angry","fearful","neutral","slow","cry","anxious",
    "kind","laugh","bright","commanding","mellow","animated",
]

STYLES = [
    "slow",
    "cry",
    "anxious",
    "kind",
    "laugh",
    "bright",
    "commanding",
    "mellow",
    "animated"
]


# ======================= CLEAN TEMPLATES =======================
TEMPLATES = [
    "A {gender} speaker delivers a {style} explanation in a clear teaching voice.",
    "This recording features a {gender} voice with a {style} speaking style.",
    "A {style} narration presented by a {gender} teacher.",
    "A {gender} educator explains the topic using a {style} tone.",
    "A clear and {style} explanation spoken by a {gender} voice.",
    "This audio contains a {gender} speaker using a {style} delivery.",
    "A {style} teaching narration performed by a {gender} individual.",
    "A professional {gender} voice speaking in a {style} manner.",
    "A calm and informative {style} explanation from a {gender} speaker.",
    "A {gender} teacher presents the topic with a {style} approach.",
    "A {style} educational explanation delivered by a {gender} voice.",
    "A natural {gender} voice expressing a {style} teaching style.",
    "A {gender} narrator speaks with a {style} tone for learning purposes.",
    "A friendly {style} explanation provided by a {gender} speaker.",
    "A focused {style} teaching voice from a {gender} educator.",
    "This sample includes a {gender} voice using a {style} narration style.",
    "A structured {style} explanation spoken by a {gender} teacher.",
    "A {gender} speaker communicates the lesson in a {style} way.",
    "An educational {style} narration performed by a {gender} voice.",
    "A confident {gender} speaker delivering a {style} explanation.",
    "A simple and {style} teaching narration from a {gender} educator.",
    "A {gender} instructional voice with a {style} expression.",
    "A clear {style} lesson explained by a {gender} speaker.",
    "A composed {gender} voice presenting content in a {style} tone.",
    "A {style} learning-focused narration by a {gender} teacher.",
    "A professional educational explanation in a {style} voice by a {gender} speaker.",
    "A {gender} speaker delivers knowledge using a {style} teaching tone.",
    "A smooth and {style} explanation spoken by a {gender} voice.",
    "A {style} classroom-style narration from a {gender} educator.",
    "A direct and {style} explanation presented by a {gender} speaker.",
    "An engaging {style} presentation given by a {gender} instructor.",
    "A {gender} narrator provides a {style} breakdown of the subject matter.",
    "The {gender} voice offers a {style} and academic delivery.",
    "A precise {style} lecture spoken by a {gender} academic.",
    "In a {style} manner, the {gender} speaker guides the listener through the topic.",
    "A highly articulate {gender} voice performing a {style} narration.",
    "The {style} quality of this {gender} speaker is perfect for educational content.",
    "A {gender} speaker uses an authoritative yet {style} tone.",
    "This {style} tutorial is narrated by a steady {gender} voice.",
    "A warm {gender} speaker provides a {style} instructional overview.",
    "The audio showcases a {gender} voice with a distinct {style} cadence.",
    "An articulate {style} explanation by a {gender} voice actor.",
    "A {gender} speaker adopts a {style} persona for this educational clip.",
    "This {style} delivery is performed by a clear-spoken {gender} individual.",
    "A {style} and methodical explanation from a {gender} speaker.",
    "The {gender} educator uses a {style} rhythm throughout the recording.",
    "A well-paced {style} narration delivered by a {gender} voice.",
    "A {gender} voice guides the lesson with a {style} and clear approach.",
    "The recording captures a {gender} speaker in a {style} teaching moment.",
    "A {style} and expressive {gender} voice recounts the educational material.",
    "This {gender} speaker provides a consistent {style} flow for learning.",
    "A balanced {style} tone is used by the {gender} narrator here.",
    "An insightful {style} explanation spoken by a {gender} specialist.",
    "The {gender} speaker maintains a {style} presence throughout the audio.",
    "A clear-cut {style} teaching style from a {gender} professional.",
    "This {gender} voice sounds both helpful and {style} in its delivery.",
    "A {style} pedagogical narration by a {gender} speaker.",
    "The {gender} speaker conveys complex ideas in a {style} tone.",
    "A rhythmic and {style} explanation given by a {gender} voice.",
    "This {style} auditory lesson is presented by a {gender} teacher."
]

# ======================= GENDER MODEL (CLARITY ONLY) =======================
print(" Loading gender classification model...")
gender_model_name = "prithivMLmods/Common-Voice-Geneder-Detection"
gender_model = Wav2Vec2ForSequenceClassification.from_pretrained(gender_model_name)
gender_processor = Wav2Vec2FeatureExtractor.from_pretrained(gender_model_name)
gender_id2label = {0: "female", 1: "male"}
gender_model.eval()

def predict_gender(audio_path):
    speech, _ = librosa.load(audio_path, sr=16000)
    inputs = gender_processor(speech, sampling_rate=16000, return_tensors="pt", padding=True)
    with torch.no_grad():
        logits = gender_model(**inputs).logits
        pred = torch.argmax(logits, dim=-1).item()
    return gender_id2label[pred]

# ======================= WAV SAVER =======================
def save_wav(filename, raw_data, rate=24000):
    if isinstance(raw_data, str):
        raw_data = base64.b64decode(raw_data)
    with wave.open(filename, "wb") as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(rate)
        wf.writeframes(raw_data)

topics = [
    # --- SPACE & COSMOS ---
    "What makes planets round?", "How fast does Earth travel through space?", "Why does the Moon have phases?",
    "How do astronauts train on Earth?", "What is microgravity?", "Why do stars have different colors?",
    "How long does it take light to reach Earth?", "What happens if you cry in space?", "Why do astronauts wear helmets?",
    "What is a space vacuum?", "How do rockets escape Earth’s gravity?", "Why does the Moon always show the same side?",
    "What are meteorites made of?", "How do astronauts stay healthy in space?", "Why is the Sun so hot?",
    "What is cosmic radiation?", "How do planets form?", "Why does Mars have dust storms?", "What is a solar eclipse?",
    "How do space missions return to Earth?", "What is a dwarf planet?", "Why does Saturn have many moons?",
    "How do space telescopes work?", "What is interstellar space?", "Why is Uranus tilted?", "How do we name stars?",
    "What is a red giant star?", "How do astronauts exercise in space?", "Why does the Sun rise in the east?",
    "What is a space capsule?", "Why does space have no air?", "How do astronomers study planets?",
    "What is a gravity assist?", "Why does Mercury have extreme temperatures?", "How do we detect asteroids?",
    "What is a binary star?", "Why does the Moon affect tides?", "How do spacecraft land on other planets?",
    "What is a cosmic ray?", "Why is Jupiter’s gravity so strong?", "What is the Oort Cloud?",
    "How do black holes form?", "What is a nebula?", "Why is Venus the hottest planet?",
    "How do we measure distances in space?", "What is dark matter?", "How does a supernova happen?",
    "What is the Big Bang theory?", "Why is Pluto not a planet anymore?", "How do we search for alien life?",
    "What are the rings of Saturn made of?", "How does the ISS stay in orbit?", "What is a pulsar?",
    "Why is the Milky Way called that?", "How do we launch satellites?", "What is a Lagrange point?",
    "How do solar flares affect Earth?", "What is the heliosphere?", "Why do comets have tails?",
    "How do we see the edge of the universe?",

    # --- EARTH & GEOLOGY ---
    "How old is the Earth?", "Why does the Earth spin?", "How do continents move?", "What causes hot spots on Earth?",
    "Why do volcanoes form near oceans?", "How do mountains rise?", "Why does sand feel hot?",
    "How do rivers create valleys?", "What makes soil fertile?", "Why do rocks have layers?",
    "How do earthquakes release energy?", "Why do deserts expand?", "How does wind cause erosion?",
    "What makes clay sticky?", "Why do some rocks float?", "How do natural arches form?",
    "Why does land sink in some areas?", "How do caves get their shapes?", "Why do beaches have waves?",
    "How does ice shape landscapes?", "What causes landslides?", "Why do rivers flood?",
    "How does sediment build deltas?", "Why does lava glow?", "How do hot springs stay warm?",
    "Why do cliffs collapse?", "How does Earth recycle rocks?", "What causes sinkholes?",
    "Why do volcanoes sleep for years?", "How do rocks become smooth?", "Why do some mountains erupt?",
    "How do glaciers carve land?", "Why is Earth’s mantle hot?", "How do mineral crystals form?",
    "Why does land crack in droughts?", "How do fault lines form?", "Why do rocks sparkle?",
    "How does erosion affect farms?", "Why do islands sink over time?", "How does Earth cool itself?",
    "What is the Earth's core made of?", "How do geysers work?", "Why is the ocean floor spreading?",
    "What are tectonic plates?", "How do fossil fuels form?", "Why are some mountains jagged?",
    "How do earthquakes cause tsunamis?", "What is the ring of fire?", "How does the magnetic pole flip?",
    "What is a stalactite?", "How do diamonds reach the surface?", "Why is topsoil so important?",
    "How do canyons form?", "What is metamorphic rock?", "How do we measure earthquake magnitude?",
    "Why are there different types of sand?", "What causes a volcanic winter?", "How do coal mines form?",
    "What is obsidian?", "How does permafrost melt?",

    # --- ANIMALS & WILDLIFE ---
    "How do animals adapt to cold?", "Why do birds build nests?", "How do fish breathe underwater?",
    "Why do animals migrate?", "How do insects survive winter?", "Why do mammals have fur?",
    "How do animals communicate silently?", "Why do some animals glow?", "How do predators hunt?",
    "Why do animals camouflage?", "How do baby animals learn?", "Why do animals groom themselves?",
    "How do animals find water?", "Why do reptiles bask in the sun?", "How do animals balance?",
    "Why do animals play fight?", "How do animals remember paths?", "Why do animals live in groups?",
    "How do animals sense storms?", "Why do animals have instincts?", "How do birds build strong nests?",
    "Why do animals mark territory?", "How do animals survive drought?", "Why do animals change color?",
    "How do animals heal wounds?", "Why do animals molt?", "How do nocturnal animals see?",
    "Why do animals stretch?", "How do animals smell danger?", "Why do animals nap often?",
    "How do animals use whiskers?", "Why do animals groom others?", "How do animals find mates?",
    "Why do animals defend babies?", "How do animals learn tricks?", "Why do animals have claws?",
    "How do animals share food?", "Why do animals migrate long distances?", "How do animals stay cool?",
    "Why do animals sleep differently?", "How do bees make honey?", "Why do spiders spin webs?",
    "How do bats use echolocation?", "Why do elephants have trunks?", "How do snakes move without legs?",
    "Why do kangaroos have pouches?", "How do dolphins communicate?", "Why do penguins huddle?",
    "How do butterflies migrate?", "Why do owls turn their heads?", "How do octopuses change texture?",
    "Why do wolves howl?", "How do ants build colonies?", "Why do sharks never stop swimming?",
    "How do turtles find the ocean?", "Why do peacocks have big tails?", "How do frogs catch flies?",
    "Why do camels have humps?", "How do squirrels find buried nuts?", "Why do cats purr?",

    # --- HUMAN BODY ---
    "Why does our heart never rest?", "How does blood carry oxygen?", "Why do muscles get sore?",
    "How does the brain learn?", "Why do we need sleep?", "How do bones heal themselves?",
    "Why do we breathe faster when scared?", "How does digestion start?", "Why does our body need water?",
    "How does skin protect us?", "Why do we feel hungry?", "How does the brain control movement?",
    "Why do we sweat?", "How does the body fight viruses?", "Why do eyes blink automatically?",
    "How does the body balance?", "Why do we feel emotions?", "How does memory form?",
    "Why do we feel pain?", "How does the body grow taller?", "Why does hair fall out?",
    "How does the body repair cells?", "Why do we feel fear?", "How does hearing work?",
    "Why do we need vitamins?", "How does taste work?", "Why do we need oxygen?",
    "How does the immune system learn?", "Why does the body need salt?", "How do nerves send signals?",
    "Why do we yawn together?", "How does vision form images?", "Why do we feel stress?",
    "How does posture work?", "Why does the heart beat rhythmically?", "How does the body remove waste?",
    "Why do we feel tired after eating?", "How does the brain control speech?", "Why do we get muscle cramps?",
    "How does the body stay warm?", "How do lungs expand?", "Why do we have eyebrows?",
    "How does the liver clean blood?", "Why do we get goosebumps?", "How does the pancreas manage sugar?",
    "Why do we have 10 fingers?", "How do kidneys filter water?", "Why do we dream in sleep?",
    "How does the spine support us?", "Why do we have different blood types?", "How does the body heal a cut?",
    "Why do we get fevers?", "How do ears help us balance?", "Why do we sneeze?",
    "How does the stomach not digest itself?", "Why do we have baby teeth?", "How does the voice work?",
    "Why do we shiver?", "How do fingernails grow?", "Why do we have a belly button?",

    # --- PHYSICS ---
    "Why does light travel so fast?", "How does motion start?", "Why does sound fade with distance?",
    "How does gravity pull objects?", "Why does friction create heat?", "How does energy transfer?",
    "Why does pressure increase underwater?", "How does momentum work?", "Why do objects fall straight down?",
    "How does acceleration change speed?", "Why does light bend?", "How does reflection work?",
    "Why does electricity flow?", "How does force change motion?", "Why does vibration create sound?",
    "How does heat spread?", "Why do waves carry energy?", "How does magnetism attract metal?",
    "Why does mass matter?", "How does inertia work?", "Why do objects resist motion?",
    "How does tension work?", "Why does sound travel faster in solids?", "How does pressure move liquids?",
    "Why do objects compress?", "How does energy transform?", "Why does speed depend on mass?",
    "How does equilibrium work?", "Why do objects oscillate?", "How does resonance work?",
    "Why does air resist motion?", "How does torque rotate objects?", "Why do objects spin?",
    "How does balance work?", "Why does temperature rise?", "How does light scatter?",
    "Why does sound vibrate air?", "How does force multiply?", "Why do objects bounce?",
    "How does pressure equalize?", "What is a vacuum?", "How does nuclear fusion work?",
    "Why does a compass point North?", "What is static electricity?", "How do lasers work?",
    "What is the Doppler effect?", "Why do rainbows have 7 colors?", "How does a prism work?",
    "What is centrifugal force?", "Why does heat rise?", "How does a pulley work?",
    "What is kinetic energy?", "How does a magnifying glass work?", "What is potential energy?",
    "Why is the speed of sound slower than light?", "How does a transistor work?", "What is refraction?",
    "How do X-rays see through skin?", "What is a conductor?", "How does gravity work in a vacuum?",

    # --- TECHNOLOGY & ENGINEERING ---
    "How do machines reduce effort?", "Why do gears change speed?", "How do engines convert fuel?",
    "Why do structures need supports?", "How do circuits control power?", "Why do machines vibrate?",
    "How do sensors detect motion?", "Why do motors spin?", "How do robots follow commands?",
    "Why do machines need cooling?", "How do levers lift weight?", "Why do pulleys reduce force?",
    "How do engines stay efficient?", "Why do devices need processors?", "How do signals transmit data?",
    "Why do machines need calibration?", "How do brakes control motion?", "Why do machines need lubrication?",
    "How do computers store data?", "Why do devices need updates?", "How do antennas receive signals?",
    "Why do machines wear out?", "How do sensors measure temperature?", "Why do engines need air?",
    "How do machines stay balanced?", "Why do systems need backups?", "How do chips process logic?",
    "Why do machines need power?", "How do controllers manage systems?", "Why do devices overheat?",
    "How do tools multiply force?", "Why do machines fail?", "How do mechanisms lock?",
    "Why do systems need testing?", "How do machines self-correct?", "Why do devices use software?",
    "How do signals encode data?", "Why do machines follow rules?", "How do systems synchronize?",
    "Why do machines automate tasks?", "How does GPS find your location?", "Why do airplanes have wings?",
    "How does a touchscreen work?", "Why do we use fiber optics?", "How does a 3D printer work?",
    "Why do we need Wi-Fi?", "How does a jet engine push a plane?", "Why do skyscrapers sway?",
    "How does a battery charge?", "Why do we use silicon for chips?", "How does a drone stay level?",
    "Why do submarines sink and float?", "How does a microwave cook food?", "Why do we have QR codes?",
    "How does a solar panel make power?", "Why do bridges have expansion joints?", "How does Bluetooth connect?",
    "Why do we use encryption?", "How does a hard drive spin?", "Why do cameras need lenses?",

    # --- FOOD & CHEMISTRY ---
    "Why does food change color when cooked?", "How does fermentation work?", "Why do spices smell strong?",
    "How does sugar preserve food?", "Why does fat carry flavor?", "How does heat change texture?",
    "Why does baking need timing?", "How do acids affect taste?", "Why does oil fry food?",
    "How does freezing slow bacteria?", "Why does salt draw water?", "How does caramelization work?",
    "Why does dough need kneading?", "How do enzymes change food?", "Why does food spoil?",
    "How does marinating soften meat?", "Why does food smell when hot?", "How does sugar crystallize?",
    "Why does milk curdle?", "How does cooking kill germs?", "Why do onions make you cry?",
    "How does yeast make bread rise?", "Why does chocolate melt in your mouth?", "How do bubbles form in soda?",
    "Why is honey so thick?", "How does vinegar clean things?", "Why does fruit ripen?",
    "How does caffeine keep you awake?", "Why does popcorn pop?", "How does soap remove grease?",
    "How does baking soda make cakes fluffy?", "Why does iron rust?", "How does a match ignite?",
    "Why is water the universal solvent?", "How do vitamins dissolve?", "Why does bread get moldy?",
    "How do artificial sweeteners work?", "Why is ice slippery?", "How does smoke form?",
    "Why do eggs turn solid when boiled?", "How do preservatives work?", "Why is coffee bitter?",
    "How does gelatin set?", "Why do apples turn brown?", "How does distillation work?",
    "Why does cheese melt?", "How do chili peppers burn?", "Why is salt necessary for life?",
    "How do antioxidants work?", "Why does silver tarnish?", "How does bleach whiten clothes?",
    "Why do lemons taste sour?", "How does gluten work?", "Why is olive oil healthy?",
    "How do food dyes work?", "Why does tea have tannins?", "How does a pressure cooker work?",
    "Why do we pasteurize milk?", "How do minerals get into vegetables?", "Why does garlic smell?",

    # --- ENVIRONMENT & ECOLOGY ---
    "How do ecosystems stay balanced?", "Why do species depend on each other?", "How do plants clean air?",
    "Why does pollution harm wildlife?", "How do forests control climate?", "Why do wetlands prevent floods?",
    "How does recycling save energy?", "Why does biodiversity matter?", "How do oceans regulate temperature?",
    "Why do humans impact habitats?", "How does climate affect animals?", "Why do ecosystems collapse?",
    "How do food chains work?", "Why do invasive species spread?", "How does conservation help nature?",
    "Why do coral reefs die?", "How do trees store carbon?", "Why does soil erosion matter?",
    "How does pollution spread?", "Why does water scarcity happen?", "How does composting help the soil?",
    "Why do we need bees for food?", "How does plastic affect the ocean?", "Why do glaciers melt?",
    "How does the ozone layer protect us?", "Why do we use renewable energy?", "How does rain wash away pollutants?",
    "Why do urban areas feel hotter?", "How do seeds travel long distances?", "Why is clean water a human right?",
    "What is the greenhouse effect?", "How do tidal waves form?", "Why is the Amazon Rainforest important?",
    "How does acid rain form?", "Why do we protect endangered species?", "How do mountains affect weather?",
    "What is a carbon footprint?", "How do mangroves protect coasts?", "Why is overfishing a problem?",
    "How do desert animals find water?", "Why do we have national parks?", "How does desertification happen?",
    "What is sustainable farming?", "How do solar farms help?", "Why is wind energy clean?",
    "How do worms improve soil?", "Why do we need to reduce waste?", "How does groundwater get stored?",
    "Why do rivers run dry?", "How do wildflowers help insects?", "What is an ecological niche?",
    "How does nature recover after fire?", "Why do we have different biomes?", "How do volcanic islands form life?",
    "Why is the Arctic melting?", "How do we measure air quality?", "Why do we use biofuels?",
    "How does logging affect weather?", "Why do we need to save the oceans?", "How do urban gardens help?",

    # --- HISTORY & CIVILIZATION ---
    "Why did ancient people build pyramids?", "How did the Silk Road connect the world?", "Why did the Roman Empire collapse?",
    "How was paper invented?", "Why did explorers sail across oceans?", "How did the printing press change history?",
    "Why did the Industrial Revolution start?", "How were ancient cities designed?", "Why did civilizations settle near rivers?",
    "How did humans discover fire?", "Why did the Renaissance happen?", "How did codes and ciphers win wars?",
    "Why did knights wear armor?", "How did the Great Wall of China start?", "Why did the Maya civilization vanish?",
    "How did early humans migrate?", "Why did the Vikings travel so far?", "How was the first calendar made?",
    "Why did castles have moats?", "How did the steam engine change travel?", "Why did the Olympics begin?",
    "How did writing systems evolve?", "Why did nations sign the Magna Carta?", "How did the telephone change communication?",
    "Why was the Eiffel Tower built?", "How did agriculture lead to cities?", "Why did ancient Greeks value philosophy?",
    "How did the compass help sailors?", "Why did the Age of Enlightenment occur?", "How did railroads unify countries?",
    "Why did gold rushes happen?", "How did the Wright brothers fly?", "Why did the Apollo 11 mission matter?",
    "How did the internet begin?", "Why did the Berlin Wall fall?", "How did early doctors study the body?",
    "Why did trade routes create wealth?", "How did the Magna Carta influence laws?", "Why did the Victorian era change society?",
    "How did electricity change city life?", "How did the wheel change humanity?", "Why was the Great Library of Alexandria famous?",
    "How did gunpowder reach Europe?", "Why did the Black Death end?", "How did the Samurai live?",
    "Why did the Aztecs build floating gardens?", "How did the Phoenicians navigate?", "Why did the Crusades happen?",
    "How did the Incas build Machu Picchu?", "Why did the American Revolution start?", "How did the Cold War shape technology?",
    "Why was the Rosetta Stone important?", "How did the Mongols conquer so much?", "Why did the Titanic sink?",
    "How did the radio change war?", "Why was the Panama Canal built?", "How did ancient Egypt use the Nile?",
    "Why did the Han Dynasty expand?", "How did the French Revolution change politics?", "Why was the Taj Mahal built?",

    # --- MATHEMATICS & LOGIC ---
    "Why is zero a special number?", "How does the Fibonacci sequence appear in nature?", "Why do we use base-10 for counting?",
    "How do prime numbers protect data?", "Why is pi an infinite number?", "How does geometry help architects?",
    "Why do we use symbols for math?", "How do probability and luck differ?", "Why is the Pythagorean theorem useful?",
    "How do computers use binary?", "Why do circles have 360 degrees?", "How does algebra solve for unknowns?",
    "Why are fractals found in plants?", "How does statistics help predict trends?", "Why is the Golden Ratio beautiful?",
    "How do graphs visualize data?", "Why do we have negative numbers?", "How do decimals work?",
    "Why is symmetry important in math?", "How do logic puzzles train the brain?", "Why do we use metric and imperial units?",
    "How does calculus measure change?", "Why is the number 7 considered lucky?", "How do algorithms make decisions?",
    "Why are triangles the strongest shape?", "How do we calculate the area of a sphere?", "Why do prime numbers never end?",
    "How do ratios help in cooking?", "Why is math called the universal language?", "How do patterns help us learn?",
    "What is an imaginary number?", "How do we calculate the speed of light?", "Why are honeycombs hexagonal?",
    "How do percentages work?", "Why is the number 13 considered unlucky?", "How do we find the volume of a cone?",
    "What is a paradox?", "How do Venn diagrams help logic?", "Why do we use X as a variable?",
    "How does interest grow over time?", "What is a matrix?", "How do we measure angles?",
    "Why is a square a rectangle?", "How do we use math in music?", "What is the Monty Hall problem?",
    "How do prime factors work?", "Why is logic important for coding?", "How do we estimate large numbers?",
    "What is game theory?", "How do coordinates find a point?", "Why is math essential for physics?",
    "How do we calculate odds?", "What is the difference between mean and median?", "How does a compass draw a circle?",
    "Why do we use Roman numerals?", "How do exponents work?", "What is a proof in math?",
    "How do we use math to predict weather?", "Why are some numbers irrational?", "How does a calculator think?",

    # --- WEATHER & METEOROLOGY ---
    "What causes thunder and lightning?", "How do clouds form?", "Why is the sky blue?",
    "How do hurricanes get their names?", "Why does it smell like rain?", "How do rainbows form?",
    "Why is it colder at the poles?", "How do tornadoes spin?", "Why does wind blow?",
    "How does humidity affect temperature?", "Why does snow have different shapes?", "How do seasons change?",
    "Why is fog common in the morning?", "How do we predict the weather?", "Why are sunsets red and orange?",
    "How does air pressure affect us?", "Why do monsoons happen?", "How does the Jet Stream move?",
    "Why is hail different from snow?", "How do heat waves occur?", "Why does the wind howl?",
    "How do deserts stay dry?", "Why do clouds have different heights?", "How does lightning strike the ground?",
    "Why is it humid in the tropics?", "How does a barometer work?", "Why do we see our breath in winter?",
    "How do thunderstorms end?", "Why does the wind change direction?", "How does a drought start?",
    "What is a cold front?", "How do snowflakes form symmetry?", "Why is the eye of a hurricane calm?",
    "How does el Niño affect weather?", "Why is lightning so hot?", "How do we measure wind speed?",
    "What is the dew point?", "Why does thunder rumble?", "How does a rain gauge work?",
    "Why is the sky dark during a storm?", "How does a blizzard form?", "Why do clouds move?",
    "What is a trade wind?", "How does smog form?", "Why do we have rainbow halos?",
    "How does a heat dome work?", "Why is it warmer in valleys at night?", "How do weather balloons work?",
    "Why is some rain 'acidic'?", "How do we track hurricanes?", "What is an aurora borealis?",
    "How does the ocean affect wind?", "Why is it drier in winter?", "How do dust devils form?",
    "What is a microburst?", "How does global warming change weather?", "Why is there no wind in the doldrums?",
    "How does hail grow in clouds?", "Why do we have Indian Summers?", "How does a thermometer work?",

    # --- PLANTS & BOTANY ---
    "How do plants eat sunlight?", "Why do leaves change color in fall?", "How do trees communicate through roots?",
    "Why do flowers have scents?", "How do cactus survive without water?", "Why do some plants eat insects?",
    "How do seeds know when to grow?", "Why do trees have bark?", "How does water travel up a tall tree?",
    "Why do weeds grow so fast?", "How do mushrooms differ from plants?", "Why do flowers bloom at different times?",
    "How do plants protect themselves?", "Why do roots grow downward?", "How do fruits develop?",
    "Why are some plants poisonous?", "How do mosses grow without soil?", "Why do leaves have veins?",
    "How do plants help the soil?", "Why do some trees stay green all year?", "How does pollination work?",
    "Why do some trees live for thousands of years?", "How do vines climb walls?", "Why do fruits have seeds?",
    "How do plants breathe?", "Why is chlorophyll green?", "How do trees make oxygen?",
    "Why do sunflowers follow the sun?", "How do orchids grow on trees?", "Why do pinecones open?",
    "How does grass survive being stepped on?", "Why do some plants flower at night?", "How do trees heal their bark?",
    "Why do seeds have hard shells?", "How do water lilies float?", "Why do we have different types of wood?",
    "How do desert plants bloom quickly?", "Why do leaves fall off?", "How do we breed new flowers?",
    "Why are some fruits sweet and others sour?",

    # --- PSYCHOLOGY & THE MIND ---
    "Why do we dream?", "How do habits form?", "Why do we feel deja vu?", "How does music affect mood?",
    "Why do we laugh?", "How do first impressions work?", "Why do we forget things?",
    "How does color influence our feelings?", "Why do we fear the dark?", "How does meditation calm the brain?",
    "Why do we procrastinate?", "How do we recognize faces?", "Why do some people love spicy food?",
    "How does exercise improve mental health?", "Why do we cry when we are happy?", "How do we learn a new language?",
    "Why do we feel butterflies in our stomach?", "How does focus work?", "Why do we enjoy scary movies?",
    "How does confidence help us succeed?", "Why do we feel empathy?", "How does the brain process humor?",
    "Why do we have different personalities?", "How does sleep help memory?", "Why do we feel bored?",
    "How do social groups influence us?", "Why do we like certain smells?", "How does the brain handle stress?",
    "Why do we get song stuck in our head?", "How does a placebo work?", "Why do we feel shy?",
    "How do we make decisions?", "Why do we crave sugar?", "How does childhood shape us?",
    "Why do we feel regret?", "How do we perceive time?", "Why do we get 'hangry'?",
    "How does praise motivate us?", "Why do we mimic others?", "How do we overcome fears?",

    # --- ARCHITECTURE & BUILDINGS ---
    "How do skyscrapers stay up?", "Why do arches support weight?", "How do we build underwater?",
    "Why do old buildings have thick walls?", "How do we make glass buildings strong?", "Why are domes used in cathedrals?",
    "How do earthquake-proof buildings work?", "Why do we use steel in construction?", "How do tunnels not collapse?",
    "Why are roofs slanted?", "How do we build bridges over wide rivers?", "Why do stairs have specific heights?",
    "How do elevators move safely?", "Why do we use brick for houses?", "How do we cool large stadiums?",
    "Why are lighthouses round?", "How do we insulate homes?", "Why do skyscrapers have 'tuned mass dampers'?",
    "How do we restore ancient ruins?", "Why do we use concrete?", "How do we build dams?",
    "Why do some floors feel bouncy?", "How do we design acoustics in theaters?", "Why are some cities built on grids?",
    "How do we build in freezing weather?", "Why are stained glass windows iconic?", "How do we move whole buildings?",
    "Why do we use scaffolding?", "How do we test soil before building?", "Why do we have basements?",

    # --- DIGITAL WORLD & COMPUTERS ---
    "How does the internet travel under the sea?", "What is a computer virus?", "How does a mouse move the cursor?",
    "Why do we use passwords?", "How does 'the cloud' store data?", "Why is the internet so fast?",
    "How does a computer screen show colors?", "What is Artificial Intelligence?", "How do websites remember you?",
    "Why do we need computer cooling fans?", "How does a search engine find results?", "What is a bit and a byte?",
    "Why do we use 'https'?", "How do emojis work?", "What is a blockchain?", "How does a motherboard connect parts?",
    "Why do we have different programming languages?", "How does voice recognition work?",
    "What is the difference between RAM and ROM?", "How do video games render graphics?",
    "Why do we use cookies on websites?", "How does a router send data?", "What is an IP address?",
    "How does a firewall protect us?", "Why do we use 4G and 5G?", "How does a digital camera store photos?",
    "What is a pixel?", "How do speakers make sound?", "Why do we need to 'eject' USB drives?",
    "How does a laptop battery last long?",

    # --- TRANSPORTATION & VEHICLES ---
    "How do airplanes fly upside down?", "Why do boats float?", "How do trains stay on tracks?",
    "Why do car tires have treads?", "How do helicopters hover?", "Why do we use gears in bikes?",
    "How does a hot air balloon rise?", "Why do rockets need stages?", "How do electric cars work?",
    "Why do race cars have spoilers?", "How do cruise ships stay balanced?", "Why do we use asphalt for roads?",
    "How do traffic lights synchronize?", "Why do airplanes fly so high?", "How do jet skis move?",
    "Why do we use tunnels for trains?", "How do bicycles stay upright?", "Why do cars have airbags?",
    "How do sailboats go against the wind?", "Why do we have different types of fuel?",
    "How do aircraft carriers work?", "Why do trucks have so many wheels?", "How do autopilots work?",
    "Why do we use maglev trains?", "How do subways get air?", "Why do we use bridges instead of ferries?",
    "How do parachutes work?", "Why do spaceships need heat shields?", "How do forklifts lift heavy loads?",
    "Why do we use tractors in farming?",

    # --- EVERYDAY MYSTERIES ---
    "How does a light switch work?", "Why do mirrors flip left and right?", "How do zippers stay closed?",
    "Why do we have 24 hours in a day?", "How does a thermostat know the temperature?", "Why does glue not stick inside the bottle?",
    "How does a thermos keep drinks hot?", "Why do pencil erasers work?", "How do we make soap?",
    "Why does metal feel cold?", "How do batteries die?", "Why do we have leap years?",
    "How does a piano make music?", "Why does rubber bounce?", "How does a ballpoint pen work?",
    "Why do we have coins?", "How does a lock and key work?", "Why do we use stamps?",
    "How does a scale weigh you?", "Why do we have different languages?", "How do we make paper?",
    "Why do we wear clothes?", "How does a candle burn?", "Why do we use blankets?",
    "How does a faucet stop water?", "Why do we have surnames?", "How does a stove cook food?",
    "Why do we use umbrellas?", "How do we make plastic?", "Why do we have flags?",

    # --- MUSIC & ART ---
    "How does a guitar make sound?", "Why do we like certain songs?", "How do we paint with oils?",
    "Why do violins have 'f-holes'?", "How do we record sound?", "Why do we use different colors in art?",
    "How does a drum vibrate?", "Why do we have 12 notes in music?", "How does a sculptor carve stone?",
    "Why do we find certain patterns beautiful?", "How do wind instruments work?", "Why do we have different art styles?",
    "How does a microphone work?", "Why do we use brushes for painting?", "How does rhythm work?",
    "Why do we have choirs?", "How do we make pottery?", "Why do we use canvases?",
    "How does a speaker produce bass?", "Why do we have museums?",

    # --- EXTRA VARIED TOPICS ---
    "How do we measure the depth of the ocean?", "Why do we use maps?", "How do we find oil underground?",
    "Why do we have different seasons?", "How do we make glass?", "Why do we have holidays?",
    "How do we build roads?", "Why do we have police?", "How do we make money?",
    "Why do we have schools?", "How do we keep food cold?", "Why do we have clocks?",
    "How do we fly?", "Why do we have cities?", "How do we clean water?",
    "Why do we have books?", "How do we build cars?", "Why do we have movies?",
    "How do we grow food?", "Why do we have electricity?", "How do we use satellites?",
    "Why do we have medicine?", "How do we use fire?", "Why do we have language?",
    "How do we make steel?", "Why do we have art?", "How do we study history?",
    "Why do we have science?", "How do we use logic?", "Why do we have math?",
    "How do we observe the stars?", "Why do we have oxygen in our atmosphere?", "How do we generate electricity?",
    "Why do we use rubber for tires?", "How do we make silk?", "Why do we use salt on icy roads?",
    "How do we track time?", "Why do we have names for months?", "How do we make cement?",
    "Why do we use aluminum for planes?", "How do we protect ourselves from the sun?", "Why do we have gravity?",
    "How do we see colors?", "Why do we have hair?", "How do we hear sounds?",
    "Why do we feel heat?", "How do we smell things?", "Why do we have skin?",
    "How do we move our arms?", "Why do we have a heart?", "How do we digest food?",
    "Why do we sleep?", "How do we breathe?", "Why do we grow?",
    "How do we learn?", "Why do we feel happy?", "Why do we feel sad?",
    "How do we think?", "Why do we have memory?", "How do we talk?",
    "Why do we live in houses?", "How do we cook?", "Why do we use tools?",
    "How do we travel?", "Why do we have seasons?", "How do we count?",
    "Why do we have money?", "How do we trade?", "Why do we have laws?",
    "How do we build bridges?", "Why do we have wheels?", "How do we write?",
    "Why do we have paper?", "How do we print?", "Why do we have the internet?",
    "How do we use computers?", "Why do we have phones?", "How do we send mail?",
    "Why do we have clothes?", "How do we make shoes?", "Why do we have music?",
    "How do we dance?", "Why do we play sports?", "How do we make movies?",
    "Why do we have theaters?", "How do we paint?", "Why do we draw?",
    "How do we build skyscrapers?", "Why do we have gardens?", "How do we use energy?",
    "Why do we have atoms?", "How do we see light?", "Why do we have electricity?",
    "How do we use magnetism?", "Why do we have planets?", "How do we study space?",
    "Why do we have the moon?", "How do we have tides?", "Why do we have volcanoes?",
    "How do we have earthquakes?", "Why do we have mountains?", "How do we have oceans?",
    "Why do we have rivers?", "How do we have rain?", "Why do we have clouds?",
    "How do we have wind?", "Why do we have fire?", "How do we have life?",
    "Why do we have animals?", "How do we have plants?", "Why do we have trees?",
    "How do we have grass?", "Why do we have flowers?", "How do we have fruit?",
    "Why do we have vegetables?", "How do we have seeds?", "Why do we have soil?",
    "How do we have rocks?", "Why do we have minerals?", "How do we have metals?",
    "Why do we have gold?", "How do we have iron?", "Why do we have carbon?",
    "How do we have nitrogen?", "Why do we have hydrogen?", "How do we have chemistry?",
    "Why do we have physics?", "How do we have biology?", "Why do we have astronomy?",
    "How do we have geology?", "Why do we have psychology?", "How do we have sociology?",
    "Why do we have philosophy?", "How do we have logic?", "Why do we have geometry?",
    "How do we have algebra?", "Why do we have calculus?", "How do we have statistics?",
    "Why do we have probability?", "How do we have patterns?", "Why do we have shapes?",
    "How do we have numbers?", "Why do we have zero?", "How do we have infinity?",
    "Why do we have time?", "How do we have space?", "Why do we have the universe?",
]


total_samples = 1200
random.shuffle(topics)
final_topics = (topics * (total_samples // len(topics) + 1))[:total_samples]

print(" Starting dataset generation...")

# ======================= MAIN GENERATION LOOP =======================
for i, topic in enumerate(final_topics, 1):
    style = random.choice(STYLES)
    voice = random.choice(VOICES)
    template = random.choice(TEMPLATES)

    # STRICTLY assign gender from the map
    assigned_gender = VOICE_MAP[voice]

    sample_id = f"teacher_{i:04d}_{datetime.now().strftime('%Y%m%d_%H%M%S_%f')}"
    audio_path = os.path.join(VOICE_DIR, f"{sample_id}.wav")

    print(f"\n [{i}/{total_samples}] {topic} | Voice: {voice} ({assigned_gender}) | Style: {style} | id: {sample_id}")

    try:
        # 1. Generate Teacher Text
        text_resp = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=f"As a teacher feeling {style}, explain '{topic}' in 2 simple sentences for a child."
        )
        teacher_text = text_resp.text.strip()

        # 2. Use Mapped Gender for Description
        final_description = template.format(
            gender=assigned_gender,
            style=style
        )

        # 3. Generate Audio
        audio_resp = client.models.generate_content(
            model="gemini-2.5-flash-preview-tts",
            contents=f"{final_description}\n\n{teacher_text}",
            config=types.GenerateContentConfig(
                response_modalities=["AUDIO"],
                speech_config=types.SpeechConfig(
                    voice_config=types.VoiceConfig(
                        prebuilt_voice_config=types.PrebuiltVoiceConfig(voice_name=voice)
                    )
                )
            )
        )

        audio_bytes = audio_resp.candidates[0].content.parts[0].inline_data.data
        save_wav(audio_path, audio_bytes)

        # 4. RUN CLASSIFIER FOR CLARITY CHECK ONLY
        predicted_gender = predict_gender(audio_path)

        # Display check: Does the model agree with your map?
        if assigned_gender != predicted_gender:
            print(f" Clarity: Model predicted {predicted_gender}, but using Map: {assigned_gender}")
        else:
            print(f" Clarity: Model agreed with Map ({assigned_gender})")

        # 5. Save Metadata using assigned_gender (Map)
        with open(METADATA_FILE, "a") as f:
            f.write(json.dumps({
                "audio_file": f"voices/{sample_id}.wav",
                "text": teacher_text,
                "description": final_description,
                "voice_name": voice,
                "style": style,
                "topic": topic,
                "gender": assigned_gender  # ALWAYS use the Map gender
            }) + "\n")

        time.sleep(8)

    except Exception as e:
        if "quota" in str(e).lower() or "429" in str(e):
            rotate_key()
        else:
            print(f" Error: {e}")

print("\n DONE — Clean dataset with gender + style only.")